In [2]:
import pandas as pd
import geopandas as gpd
import json
import os
import folium


# List of file paths and corresponding scenarios
stop_file_paths = [
    'data/WFv9.1_OY_2023_transit_boarding_summary_segment.csv', 
    'data/WFv9.1_RTP_2050_transit_boarding_summary_segment.csv'
]
stop_scenarios = ['oy-2023', 'rtp-2050']
aliases = ['2023', 'RTP 2050']

stop_key_df = pd.read_csv('data/WFv901_Segments_STOP_KEY.csv')

stops_gdf = gpd.read_file('data/WFv901_Segments_STOP.geojson')

max_mode_df = pd.DataFrame([
    [4,1],
    [5,2],
    [6,4],
    [7,5],
    [8,6],
    [9,4]
], columns=(['Mode','ModeHier']))
max_mode_df


# Function to return buffer distance based on MaxMode
def get_buffer_distance(max_mode):
    if max_mode == 4:
        return 1/4  # 1/4 mile
    elif max_mode in [5, 6]:
        return 1/2  # 1/2 mile
    else:
        return 0  # No buffer for other modes


In [3]:
scenarios = ['oy-2023', 'oy-2023', 'oy-2023', 'rtp-2050', 'rtp-2050', 'rtp-2050',]
map_trip_orientation = ['P','A','OD', 'P','A','OD']
map_scenarios = ['oy-2023-p', 'oy-2023-a', 'oy-2023-od', 'rtp-2050-p', 'rtp-2050-a', 'rtp-2050-od',]
aliases = ['2023 Productions', '2023 Attractions', '2023 OD', '2050 RTP Productions', '2050 RTP Attractions', '2050 RTP OD',]


In [4]:
_df1 = pd.read_csv('data/OY_2023_ZoneSummary_TripsByMode.csv')
_df1['scenario'] = 'oy-2023'
_df2 = pd.read_csv('data/RTP_2050_ZoneSummary_TripsByMode.csv')
_df2['scenario'] = 'rtp-2050'

trips_by_mode_df = pd.concat([_df1,_df2])

trips_by_mode_df = trips_by_mode_df[(trips_by_mode_df['Purpose']=='All') & (trips_by_mode_df['Period']=='Dy')]

trips_by_mode_df = trips_by_mode_df[['scenario','TAZID','PA','tTrn']]

trips_by_mode_df = trips_by_mode_df.pivot(index=(['scenario','TAZID']), values='tTrn', columns='PA').reset_index()

trips_by_mode_df['OD'] = (trips_by_mode_df['P'] / 2) + (trips_by_mode_df['A'] / 2)

# show only one trip
trips_by_mode_df['P'] = (trips_by_mode_df['P'] / 2)
trips_by_mode_df['A'] = (trips_by_mode_df['A'] / 2)

display(trips_by_mode_df)
trips_by_mode_df.max()

PA,scenario,TAZID,A,P,OD
0,oy-2023,1.0,0.0,0.000,0.000
1,oy-2023,2.0,0.0,0.000,0.000
2,oy-2023,3.0,0.0,0.100,0.100
3,oy-2023,4.0,0.0,0.005,0.005
4,oy-2023,5.0,0.0,0.000,0.000
...,...,...,...,...,...
7253,rtp-2050,3625.0,0.0,0.000,0.000
7254,rtp-2050,3626.0,0.0,0.000,0.000
7255,rtp-2050,3627.0,0.0,0.000,0.000
7256,rtp-2050,3628.0,0.0,63.435,63.435


PA
scenario    rtp-2050
TAZID         3629.0
A           21288.54
P            1384.02
OD          22672.56
dtype: object

In [5]:
# Step 2: Load the GeoJSON file as a GeoDataFrame
taz_gdf = gpd.read_file('data/WFv900_TAZ.geojson')

# Step 3: Merge dfTripsByMode with the GeoDataFrame
taz_gdf['geometry'] = taz_gdf['geometry'].simplify(0.0001, preserve_topology=True)
taz_with_transit_gdf = taz_gdf.merge(trips_by_mode_df, on='TAZID', how='left')
taz_with_transit_gdf

,TAZID,DISTSML,DISTMED,DISTLRG,DISTSUPER,DSUP_NAME,CITY_NAME,PLANAREA,SUBAREAID,DSML_NAME,DMED_NAME,DLRG_NAME,SUBPLNAREA,ACRES,DEVACRES,geometry,scenario,A,P,OD
0,1,11,1,1,1,Box Elder County,Box Elder County North,WFRC,1,Small District 011,North of Brigham City,Box Elder County - WFRC,Box Elder County,582.086695,582.086695,"POLYGON ((-112.04054 41.57247, -112.04057 41.5...",oy-2023,0.0,0.000,0.000
1,1,11,1,1,1,Box Elder County,Box Elder County North,WFRC,1,Small District 011,North of Brigham City,Box Elder County - WFRC,Box Elder County,582.086695,582.086695,"POLYGON ((-112.04054 41.57247, -112.04057 41.5...",rtp-2050,0.0,0.000,0.000
2,2,11,1,1,1,Box Elder County,Box Elder County North,WFRC,1,Small District 011,North of Brigham City,Box Elder County - WFRC,Box Elder County,135.465050,134.110399,"POLYGON ((-112.04311 41.59360, -112.04553 41.5...",oy-2023,0.0,0.000,0.000
3,2,11,1,1,1,Box Elder County,Box Elder County North,WFRC,1,Small District 011,North of Brigham City,Box Elder County - WFRC,Box Elder County,135.465050,134.110399,"POLYGON ((-112.04311 41.59360, -112.04553 41.5...",rtp-2050,0.0,0.000,0.000
4,3,11,1,1,1,Box Elder County,Box Elder County North,WFRC,1,Small District 011,North of Brigham City,Box Elder County - WFRC,Box Elder County,663.377323,636.842231,"POLYGON ((-112.08483 41.59324, -112.08410 41.5...",oy-2023,0.0,0.100,0.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7087,3504,681,68,25,7,Utah County,Eastside Mountains,MAG,1,Small District 681,Provo Canyon,Utah County - East Mountains,NaN,1203.759358,0.000000,"POLYGON ((-111.57728 40.35601, -111.57696 40.3...",rtp-2050,0.0,0.000,0.000
7088,3546,903,71,25,7,Utah County,Eastside Mountains,MAG,1,Small District 903,Spanish Fork Canyon,Utah County - East Mountains,NaN,59358.376922,0.000000,"POLYGON ((-111.04618 39.89975, -110.85765 39.8...",oy-2023,0.0,0.000,0.000
7089,3546,903,71,25,7,Utah County,Eastside Mountains,MAG,1,Small District 903,Spanish Fork Canyon,Utah County - East Mountains,NaN,59358.376922,0.000000,"POLYGON ((-111.04618 39.89975, -110.85765 39.8...",rtp-2050,0.0,0.000,0.000
7090,3507,681,68,25,7,Utah County,Eastside Mountains,MAG,1,Small District 681,Provo Canyon,Utah County - East Mountains,NaN,2360.868265,708.260480,"POLYGON ((-111.56507 40.34417, -111.55612 40.3...",oy-2023,0.0,0.050,0.050


In [6]:
def extract_max_aMxM_from_file(file_path):
    # Extract the source from the filename (assuming file is named like oy-2023-... or rtp-2050-...)
    filename = os.path.basename(file_path)
    source = '-'.join(filename.split('-')[:2])

    # Open and load the JSON data from a file
    with open(file_path, 'r') as file:
        data = json.load(file)['data']

    # Initialize a list to store the results
    results = []

    # Loop through each filter in the JSON (first-level key)
    for filter_name, segments in data.items():
        # Ensure 'segments' is a dictionary
        if isinstance(segments, dict):
            # Loop through each segment (second-level key)
            for segment, values in segments.items():
                # Ensure 'values' is a dictionary and contains 'aMxM'
                if isinstance(values, dict) and 'aMxM' in values:
                    # Add the result to the list as a tuple (segment, max_aMxM, source)
                    results.append({'SEGID': segment, 'aMxM': values['aMxM'], 'scenario': source})

    return results

# Process both files
file_paths = ['data/oy-2023-j-transit-segments.json', 'data/rtp-2050-j-transit-segments.json']
all_results = []

for file_path in file_paths:
    all_results.extend(extract_max_aMxM_from_file(file_path))

# Create a DataFrame from the results
df = pd.DataFrame(all_results)


# Manually add records to the existing DataFrame
# The links in model were missing segids
manual_records = pd.DataFrame({
    'SEGID': ['WFRC_8127', 'WFRC_8339', 'WFRC_8127', 'WFRC_8339'],
    'aMxM': [5, 5, 5, 5],
    'scenario': ['oy-2023', 'oy-2023', 'rtp-2050', 'rtp-2050']
})

df = pd.concat([df, manual_records], ignore_index=True)

df


,SEGID,aMxM,scenario
0,0006_159.3,1,oy-2023
1,0006_159.6,1,oy-2023
2,0006_159.8,1,oy-2023
3,0006_160.3,1,oy-2023
4,0006_173.4,1,oy-2023
...,...,...,...
9082,UTA_9588,6,rtp-2050
9083,WFRC_8127,5,oy-2023
9084,WFRC_8339,5,oy-2023
9085,WFRC_8127,5,rtp-2050


In [7]:
# Step 1: Group by segment and source and get the max max_aMxM
grouped_df = df.groupby(['SEGID', 'scenario']).agg({'aMxM': 'max'}).reset_index()


# Step 2: Load the GeoJSON file
geojson_path = 'data/WFv901_Segments.geojson'
gdf = gpd.read_file(geojson_path)

gdf['geometry'] = gdf['geometry'].simplify(0.0001, preserve_topology=True)

# Step 3: Merge the grouped dataframe with the GeoDataFrame
# Assuming 'segment' in grouped_df corresponds to a field in the GeoJSON
# Replace 'segment_field_in_geojson' with the actual field name in the GeoJSON that matches 'segment'
transit_lines_gdf = gdf.merge(grouped_df, on='SEGID', how='left')
transit_lines_gdf = transit_lines_gdf[['scenario','SEGID','aMxM','geometry']]
transit_lines_gdf = transit_lines_gdf[~transit_lines_gdf['aMxM'].isna()]
transit_lines_gdf

,scenario,SEGID,aMxM,geometry
10,rtp-2050,0006_158.5,1.0,"LINESTRING (-111.80811 39.98277, -111.80069 39..."
11,rtp-2050,0006_158.9,1.0,"LINESTRING (-111.80069 39.97899, -111.79469 39..."
12,oy-2023,0006_159.3,1.0,"LINESTRING (-111.79469 39.97597, -111.79315 39..."
13,rtp-2050,0006_159.3,1.0,"LINESTRING (-111.79469 39.97597, -111.79315 39..."
14,oy-2023,0006_159.6,1.0,"LINESTRING (-111.78951 39.97561, -111.78525 39..."
...,...,...,...,...
6766,oy-2023,WFRC_8481,1.0,"LINESTRING (-112.00837 41.09649, -112.00930 41..."
6767,rtp-2050,WFRC_8481,2.0,"LINESTRING (-112.00837 41.09649, -112.00930 41..."
6770,oy-2023,WFRC_8484,1.0,"LINESTRING (-111.90564 40.51546, -111.90568 40..."
6771,rtp-2050,WFRC_8484,2.0,"LINESTRING (-111.90564 40.51546, -111.90568 40..."


In [8]:
# Function to extract rail stops or relevant data from the file
def extract_boardings_from_file(file_path, scenario):
    print(f"Processing file for scenario: {scenario}")

    # Read the CSV file into a DataFrame and filter the required columns
    _df = pd.read_csv(file_path)
    _df = _df[['SegID', 'Direction', 'Mode', 'Board']]  # Adjust this as needed
    _df['scenario'] = scenario  # Add scenario column

    return _df

# List to store all DataFrames
all_results = []

# Loop over file paths and corresponding scenarios
for file_path, scenario in zip(stop_file_paths, stop_scenarios):
    all_results.append(extract_boardings_from_file(file_path, scenario))

# Concatenate all DataFrames into one
boardings_df = pd.concat(all_results, ignore_index=True)


# # Manually add records to the existing DataFrame
# # The links in model were missing segids
# manual_records = pd.DataFrame({
#     'SEGID': ['WFRC_8127', 'WFRC_8339', 'WFRC_8127', 'WFRC_8339'],
#     'aMxM': [5, 5, 5, 5],
#     'source': ['oy-2023', 'oy-2023', 'rtp-2050', 'rtp-2050']
# })

#df = pd.concat([df, manual_records], ignore_index=True)

boardings_df = pd.merge(boardings_df, max_mode_df, on='Mode')

boardings_df = boardings_df[(boardings_df['Mode'].isin([7,8,9])) & (boardings_df['Board']>0)]


boardings_df


Processing file for scenario: oy-2023
Processing file for scenario: rtp-2050


,SegID,Direction,Mode,Board,scenario,ModeHier
16962,UTA_7182,WB,7.0,53.77,oy-2023,5
16963,UTA_7178,NB,7.0,11.06,oy-2023,5
16964,UTA_7174,NB,7.0,61.76,oy-2023,5
16965,UTA_7170,NB,7.0,21.57,oy-2023,5
16966,UTA_7166,NB,7.0,15.53,oy-2023,5
...,...,...,...,...,...,...
19676,0189_001.4,SB,9.0,61.91,rtp-2050,4
19677,UTA_0990,SB,9.0,224.80,rtp-2050,4
19678,undefined,SB,9.0,6.07,rtp-2050,4
19679,3033_000.0,EB,9.0,0.23,rtp-2050,4


In [9]:
stops_with_boardings_df = pd.merge(boardings_df, stop_key_df, on=(['SegID','Direction']))
stops_with_boardings_df = stops_with_boardings_df.groupby(['STOPID','scenario'], as_index=False).agg(MaxMode=('ModeHier','max'),Boardings=('Board','sum'))
stops_with_boardings_df

,STOPID,scenario,MaxMode,Boardings
0,412840_4560220,oy-2023,6,1137.60
1,412840_4560220,rtp-2050,6,5744.74
2,413240_4490620,oy-2023,5,3.37
3,413240_4490620,rtp-2050,5,121.93
4,413240_4491700,oy-2023,5,137.24
...,...,...,...,...
318,445300_4455240,rtp-2050,4,670.88
319,445300_4455680,rtp-2050,4,728.93
320,445300_4456440,oy-2023,4,244.81
321,445300_4456440,rtp-2050,4,167.12


In [17]:
# stops gdf with max mode
stops_with_max_mode_gdf = pd.merge(stops_gdf, stops_with_boardings_df, on='STOPID')
stops_with_max_mode_gdf.to_file('results/stops_with_max_mode.geojson')
stops_with_max_mode_gdf


,STOPID,TAZID,DISTSML,DISTMED,DISTLRG,DISTSUPER,DSUP_NAME,CITY_NAME,PLANAREA,SUBAREAID,geometry,scenario,MaxMode,Boardings
0,424680_4497040,1803.0,441.0,44.0,17.0,6.0,Salt Lake County,Midvale,WFRC,1.0,POINT (-111.89049 40.62076),rtp-2050,4,415.70
1,437580_4462640,2811.0,584.0,58.0,21.0,7.0,Utah County,Orem,MAG,1.0,POINT (-111.73461 40.31195),rtp-2050,4,10.73
2,438320_4462620,2811.0,584.0,58.0,21.0,7.0,Utah County,Orem,MAG,1.0,POINT (-111.72590 40.31182),rtp-2050,4,9.52
3,438580_4462620,2836.0,583.0,58.0,21.0,7.0,Utah County,Orem,MAG,1.0,POINT (-111.72284 40.31184),rtp-2050,4,62.55
4,439240_4462640,2812.0,583.0,58.0,21.0,7.0,Utah County,Orem,MAG,1.0,POINT (-111.71507 40.31207),rtp-2050,4,279.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,424220_4501380,1633.0,341.0,34.0,14.0,6.0,Salt Lake County,Murray,WFRC,1.0,POINT (-111.89645 40.65981),oy-2023,6,1464.48
319,424220_4501380,1633.0,341.0,34.0,14.0,6.0,Salt Lake County,Murray,WFRC,1.0,POINT (-111.89645 40.65981),rtp-2050,6,6729.42
320,413240_4490620,1967.0,423.0,42.0,16.0,6.0,Salt Lake County,South Jordan,WFRC,1.0,POINT (-112.02484 40.56181),oy-2023,5,3.37
321,413240_4490620,1967.0,423.0,42.0,16.0,6.0,Salt Lake County,South Jordan,WFRC,1.0,POINT (-112.02484 40.56181),rtp-2050,5,121.93


In [11]:
stops_with_max_mode_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
# Check if geometries are present
if stops_with_max_mode_gdf.is_empty.any():
    print("Some geometries are empty!")

# Check geometry validity
print(stops_with_max_mode_gdf.is_valid)

0      True
1      True
2      True
3      True
4      True
       ... 
318    True
319    True
320    True
321    True
322    True
Length: 323, dtype: bool


In [13]:
_gdf = stops_with_max_mode_gdf.set_crs(epsg=4326)

# Ensure GeoDataFrame uses an appropriate projected CRS for distance calculations
# For example, EPSG:26912 is a UTM zone 12 N
_gdf = _gdf.to_crs(epsg=26912)

# Apply buffer around points based on MaxMode
_gdf['buffered_geometry'] = _gdf.apply(lambda row: row.geometry.buffer(get_buffer_distance(row['MaxMode']) * 1609.34), axis=1)

_gdf = _gdf[~_gdf['buffered_geometry'].is_empty]

# Create a new GeoDataFrame with the buffered geometries
buffered_stops_with_max_mode_gdf = gpd.GeoDataFrame(_gdf, geometry='buffered_geometry')
buffered_stops_with_max_mode_gdf.set_crs(epsg=26912, inplace=True)

# Reset the CRS back to the original if needed (for example, if you were using WGS84/EPSG:4326)
buffered_stops_with_max_mode_gdf = buffered_stops_with_max_mode_gdf.to_crs(epsg=4326)

# Display the buffered GeoDataFrame
#buffered_stops_with_max_mode_gdf[buffered_stops_with_max_mode_gdf['MaxMode']>=4]

# Step 1: Delete the existing 'geometry' column (if needed)
buffered_stops_with_max_mode_gdf = buffered_stops_with_max_mode_gdf.drop(columns=['geometry'])

# Step 2: Rename 'buffered_geometry' to 'geometry'
buffered_stops_with_max_mode_gdf = buffered_stops_with_max_mode_gdf.rename(columns={'buffered_geometry': 'geometry'})

# Step 3: Set the new 'geometry' column as the active geometry
buffered_stops_with_max_mode_gdf = buffered_stops_with_max_mode_gdf.set_geometry('geometry')

buffered_stops_with_max_mode_gdf = buffered_stops_with_max_mode_gdf[buffered_stops_with_max_mode_gdf.geometry.type == 'Polygon']

buffered_stops_with_max_mode_gdf

,STOPID,TAZID,DISTSML,DISTMED,DISTLRG,DISTSUPER,DSUP_NAME,CITY_NAME,PLANAREA,SUBAREAID,scenario,MaxMode,Boardings,geometry
0,424680_4497040,1803.0,441.0,44.0,17.0,6.0,Salt Lake County,Midvale,WFRC,1.0,rtp-2050,4,415.70,"POLYGON ((-111.88574 40.62079, -111.88576 40.6..."
1,437580_4462640,2811.0,584.0,58.0,21.0,7.0,Utah County,Orem,MAG,1.0,rtp-2050,4,10.73,"POLYGON ((-111.72987 40.31198, -111.72989 40.3..."
2,438320_4462620,2811.0,584.0,58.0,21.0,7.0,Utah County,Orem,MAG,1.0,rtp-2050,4,9.52,"POLYGON ((-111.72116 40.31185, -111.72118 40.3..."
3,438580_4462620,2836.0,583.0,58.0,21.0,7.0,Utah County,Orem,MAG,1.0,rtp-2050,4,62.55,"POLYGON ((-111.71810 40.31187, -111.71812 40.3..."
4,439240_4462640,2812.0,583.0,58.0,21.0,7.0,Utah County,Orem,MAG,1.0,rtp-2050,4,279.66,"POLYGON ((-111.71034 40.31210, -111.71036 40.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,424220_4501380,1633.0,341.0,34.0,14.0,6.0,Salt Lake County,Murray,WFRC,1.0,oy-2023,6,1464.48,"POLYGON ((-111.88694 40.65988, -111.88697 40.6..."
319,424220_4501380,1633.0,341.0,34.0,14.0,6.0,Salt Lake County,Murray,WFRC,1.0,rtp-2050,6,6729.42,"POLYGON ((-111.88694 40.65988, -111.88697 40.6..."
320,413240_4490620,1967.0,423.0,42.0,16.0,6.0,Salt Lake County,South Jordan,WFRC,1.0,oy-2023,5,3.37,"POLYGON ((-112.01534 40.56189, -112.01538 40.5..."
321,413240_4490620,1967.0,423.0,42.0,16.0,6.0,Salt Lake County,South Jordan,WFRC,1.0,rtp-2050,5,121.93,"POLYGON ((-112.01534 40.56189, -112.01538 40.5..."


In [14]:
if False:

    import folium
    import pandas as pd

    # Define your map scenarios
    map_scenarios = ['oy-2023', 'rtp-2050']

    # Define a central dictionary for color and width mapping
    style_mapping = {
        6: {"name": "Commuter Rail", "color": "#Af2944", "width": 6},
        5: {"name": "Light Rail", "color": "#Eb672d", "width": 5},
        4: {"name": "Express Bus", "color": "#8dc348", "width": 4},
        3: {"name": "Bus Rapid Transit", "color": "#0022ff", "width": 3},
        2: {"name": "Core Bus", "color": "#1ba9e6", "width": 2},
        1: {"name": "Local Bus", "color": "#31398a", "width": 1}
    }

    def get_style_by_aMxM(aMxM):
        # Return the corresponding style or a default if not found
        return style_mapping.get(aMxM, {"color": "#000000", "width": 1})

    # Loop through each scenario and create maps
    for map_scenario in map_scenarios:

        # Generate the custom legend dynamically using the style mapping
        legend_html = '''
        <div style="position: fixed;
            bottom:50px;left:50px;width:200px;height:200px; 
            background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
            &nbsp; <b>Transit Mode</b> <br>
        '''
        for aMxM, style in style_mapping.items():
            legend_html += f'&nbsp; <i style="background:{style["color"]};width:20px;height:{style["width"]}px;display:inline-block;"></i> {style["name"]}<br>'

        legend_html += '''
            &nbsp; <br>
            &nbsp; <i style="background:grey;width:20px;height:1px;display:inline-block;"></i> Station Buffers<br>
        '''

        legend_html += '</div>'

        # Step 4: Plot with Folium
        # Create a map centered at Murray, Utah
        m = folium.Map(location=[40.6669, -111.8870], zoom_start=11)

        # Add a grey tile layer
        folium.TileLayer('CartoDB positron', name="Grey Background").add_to(m)

        # Filter the data for the current scenario
        map_transit_gdf = taz_with_transit_gdf[taz_with_transit_gdf['scenario'] == map_scenario]

        # Add the Choropleth layer to the Folium map
        choropleth = folium.Choropleth(
            geo_data=map_transit_gdf.to_json(),  # Convert GeoDataFrame to GeoJSON
            name="Transit Trips",
            data=map_transit_gdf,
            columns=['TAZID', 'tTrn'],  # Columns to use for mapping
            key_on="feature.properties.TAZID",  # Match the TAZID in GeoJSON
            fill_color="Greens",  # Green color scale
            fill_opacity=0.5,
            line_opacity=0,
            legend_name="Transit Trips",
            threshold_scale=[0, 100, 300, 1000, 1500, 3000]
        ).add_to(m)

        # Apply zIndex for the TAZ polygons
        for feature in choropleth.geojson.data['features']:
            feature['properties']['zIndex'] = 500  # Define zIndex for polygons

        # Add a GeoJson layer on top of the choropleth for tooltips
        folium.GeoJson(
            map_transit_gdf.to_json(),  # Use the same GeoDataFrame or GeoJSON file
            name="Transit Zone Tooltips",
            tooltip=folium.GeoJsonTooltip(
                fields=["TAZID", "tTrn"],  # Fields to display in the tooltip
                aliases=["TAZ ID:", "Transit Trips:"],  # Alias names for the tooltip
                localize=True
            ),
            style_function=lambda x: {
                'fillOpacity': 0,  # Make the fill fully transparent
                'color': 'transparent',  # No border color
            }
        ).add_to(m)

        # STOP BUFFER LAYER
        map_buffered_stops_with_max_mode_gdf = buffered_stops_with_max_mode_gdf[buffered_stops_with_max_mode_gdf['scenario'] == map_scenario]

        # Define a style function for the polygons with no fill
        def style_function(feature):
            return {
                'fillOpacity': 0,  # No fill
                'color': 'grey',   # Outline color for the polygon
                'weight': 1,       # Outline thickness
                'zIndex': 20000  
            }

        # Add the GeoJSON layer to the Folium map
        folium.GeoJson(
            map_buffered_stops_with_max_mode_gdf.to_json(),
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(fields=['STOPID','Boardings'], aliases=['Segment', 'Max aMxM'])
        ).add_to(m)

        # TRANSIT LINES LAYER
        map_transit_lines_gdf = transit_lines_gdf[transit_lines_gdf['scenario'] == map_scenario]

        # Add the GeoDataFrame to the folium map with the symbology
        for _, row in map_transit_lines_gdf.iterrows():

            if pd.isna(row['aMxM']):
                continue  # Skip this row if 'aMxM' is NaN

            style = get_style_by_aMxM(row['aMxM'])  # Ensure you're using the correct column ('aMxM')

            # Create a GeoJson feature for each row
            folium.GeoJson(
                {
                    'type': 'Feature',
                    'geometry': row['geometry'].__geo_interface__,  # Convert geometry to geo_interface format
                    'properties': {  # Add properties to the GeoJson
                        'SEGID': row['SEGID'],  # Replace 'SEGID' with the actual column name in your GeoDataFrame
                        'aMxM': row['aMxM']
                    }
                },
                style_function=lambda x, style=style: {
                    'color': style['color'],
                    'weight': style['width'],
                    'opacity': 0.95,
                    'zIndex': 10000  # Ensure polylines appear above polygons
                },
                tooltip=folium.GeoJsonTooltip(fields=['SEGID', 'aMxM'], aliases=['Segment', 'Max aMxM']),
            ).add_to(m)

        # Add the custom legend to the map
        m.get_root().html.add_child(folium.Element(legend_html))

        # Save the map to an HTML file (change the filename as needed)
        m.save(f'results/map_{map_scenario}.html')


In [18]:
# Define a central dictionary for color and width mapping (assumed you have this)
style_mapping = {
    6: {"name": "Commuter Rail"    , "color": "#Af2944", "width": 6},
    5: {"name": "Light Rail"       , "color": "#Eb672d", "width": 5},
    4: {"name": "Express Bus"      , "color": "#8dc348", "width": 4},
    3: {"name": "Bus Rapid Transit", "color": "#0022ff", "width": 3},
    2: {"name": "Core Bus"         , "color": "#1ba9e6", "width": 2},
    1: {"name": "Local Bus"        , "color": "#31398a", "width": 1}
}


def get_style_by_aMxM(aMxM):
    # Return the corresponding style or a default if not found
    return style_mapping.get(aMxM, {"color": "#000000", "width": 1})


# Create an empty string to store the HTML content
tabs_html = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Zones of Transit Trips Productions, Attractions, and Origin-Destingations</title>
    <style>
        h1, h2, h3, h4, h5, h6 {
            font-family: 'Roboto', sans-serif;
        }

        /* Style for the tabs */
        .tab {
            overflow: hidden;
            border: 1px solid #ccc;
            background-color: #f1f1f1;
            display: flex;
        }

        /* Style for tab links */
        .tab button {
            background-color: inherit;
            float: left;
            border: none;
            outline: none;
            cursor: pointer;
            padding: 14px 16px;
            transition: 0.3s;
            font-size: 17px;
        }

        /* Change background color of tab links on hover */
        .tab button:hover {
            background-color: #ddd;
        }

        /* Create an active/current tab link */
        .tab button.active {
            background-color: #ccc;
        }

        /* Flex layout for full-height usage */
        html, body {
            height: 100%;
            margin: 0;
            display: flex;
            flex-direction: column;
        }

        /* Ensure the container takes the remaining height */
        #map-container {
            flex: 1;
            display: flex;
            flex-direction: column;
            position: relative;
            transition: margin-right 0.5s;
            margin-right: 250px;  /* Add margin to account for open sidebar */
        }

        /* Style the tab content (hidden by default) */
        .tabcontent {
            flex: 1;
            display: none;
            border: 1px solid #ccc;
            border-top: none;
            overflow: hidden;
            flex-direction: column;
            transition: width 0.5s;
        }

        /* Show the tab content when active */
        .tabcontent.active {
            display: flex;  /* Use flexbox to allow maps to take up remaining space */
            width: calc(100% - 250px);  /* Reduce width by the sidebar width */
        }

        /* Map container flexes to take up remaining space */
        .map {
            flex: 1;
        }

        /* Sidebar styling */
        .sidebar {
            height: 100%;
            width: 250px;  /* Open by default */
            position: fixed;
            right: 0;
            top: 0;
            background-color: #f1f1f1;
            overflow-x: hidden;
            transition: 0.5s;
            padding-top: 60px;
            border-left: 1px solid #ccc;
            z-index: 9999; /* Ensure it stays above the map */
        }

        .sidebar-content {
            padding: 20px;
            font-size: 14px;
        }

        /* Close button inside sidebar */
        .sidebar .closebtn {
            position: absolute;
            top: 0;
            right: 0;
            font-size: 36px;
            margin-right: 15px;
        }

        /* Button to open/close sidebar */
        .openbtn {
            position: absolute;
            top: 10px;
            right: 10px;
            background-color: #111;
            color: white;
            padding: 10px;
            border: none;
            cursor: pointer;
        }

        .openbtn:hover {
            background-color: #444;
        }
    </style>

</head>
<body>

<h2>&nbsp;<strong>Zones of Transit Trips Productions, Attractions, and Origin-Destingations</strong></h2>

<div class="tab">
'''

# Add the tab buttons using zip to combine map_scenarios and aliases
for i, (map_scenario, scenario, trip_orientation, alias) in enumerate(zip(map_scenarios, scenarios, map_trip_orientation, aliases)):
    tabs_html += f'<button class="tablinks" onclick="openScenario(event, \'{map_scenario}\')"'
    
    # Add the 'active' class to the first tab
    if i == 0:
        tabs_html += ' class="active"'
    
    tabs_html += f'>{alias}</button>\n'

tabs_html += '''
</div>

<!-- Sidebar for the legend -->
<div id="mySidebar" class="sidebar">
    <div class="sidebar-content">
        <a href="javascript:void(0)" class="closebtn" onclick="toggleSidebar()">&times;</a>

        <h3>Legend</h3>
        <br/>
        <h4>Transit Routes</h4>
'''

# Add the legend items dynamically from style_mapping
for aMxM, style in style_mapping.items():
    tabs_html += f'<p><span style="background-color:{style["color"]};width:40px;height:{style["width"]}px;display:inline-block;vertical-align:middle;"></span> {style["name"]}</p>\n'

tabs_html += '''
        <br>
        <i style="background:grey;width:40px;height:1px;display:inline-block;"></i> Station Buffers<br>
'''

tabs_html += '''
    </div>
</div>

<!-- Button to toggle the sidebar -->
<button class="openbtn" onclick="toggleSidebar()">☰ Legend</button>

'''
# Dictionary to store each map by scenario
maps = {}
map_ids = []


# Create the content for each map scenario in a separate div
for i, (map_scenario, scenario, trip_orientation, alias) in enumerate(zip(map_scenarios, scenarios, map_trip_orientation, aliases)):

    # Start the tab content div
    tabs_html += f'<div id="{map_scenario}_container" class="tabcontent {"active" if i == 0 else ""}">\n'

    # Create a Folium map for each scenario with a unique map ID
    # Instead of reusing 'm', create a new entry in the dictionary for each map
    maps[map_scenario] = folium.Map(location=[40.6669, -111.8870], zoom_start=11)

    # Add a grey tile layer
    folium.TileLayer('CartoDB positron', name="Grey Background").add_to(maps[map_scenario])

    # TRANSIT TRIPS LAYER ########################################################################################

    # Filter the data for the current scenario
    map_transit_gdf = taz_with_transit_gdf[taz_with_transit_gdf['scenario'] == scenario]

    # Add the Choropleth layer to the Folium map
    folium.Choropleth(
        geo_data=map_transit_gdf.to_json(),  # Convert GeoDataFrame to GeoJSON
        name="Transit Trips",
        data=map_transit_gdf,
        columns=['TAZID', trip_orientation],  # Columns to use for mapping
        key_on="feature.properties.TAZID",  # Match the TAZID in GeoJSON
        fill_color="Greens",  # Green color scale
        fill_opacity=0.75,
        line_opacity=0.05,
        line_width=0.1,
        legend_name="Transit Trips",
        threshold_scale=[0, 50, 100, 500, 1000, 5000, 10000, 25000]
    ).add_to(maps[map_scenario])

    ## Add a GeoJson layer for tooltips
    #folium.GeoJson(
    #    map_transit_gdf.to_json(),  # Use the same GeoDataFrame or GeoJSON file
    #    name="Transit Zone Tooltips",
    #    tooltip=folium.GeoJsonTooltip(
    #        fields=["TAZID", "tTrn"],  # Fields to display in the tooltip
    #        aliases=["TAZ ID:", "Transit Trips:"],  # Alias names for the tooltip
    #        localize=True
    #    ),
    #    style_function=lambda x: {
    #        'fillOpacity': 0,  # Make the fill fully transparent
    #        'color': 'transparent',  # No border color
    #    }
    #).add_to(maps[map_scenario])

    # STOP BUFFER LAYER #########################################################################################
    map_buffered_stops_with_max_mode_gdf = buffered_stops_with_max_mode_gdf[buffered_stops_with_max_mode_gdf['scenario'] == scenario]

    # Define a style function for the polygons with no fill
    def style_function(feature):
        return {
            'fillOpacity': 0,  # No fill
            'color': 'grey',   # Outline color for the polygon
            'weight': 1,       # Outline thickness
            'zIndex': 20000  
        }

    # Add the GeoJSON layer to the Folium map
    folium.GeoJson(
        map_buffered_stops_with_max_mode_gdf.to_json(),
        style_function=style_function,
        #tooltip=folium.GeoJsonTooltip(fields=['STOPID','Boardings'], aliases=['Segment', 'Max aMxM'])
    ).add_to(maps[map_scenario])

    # TRANSIT LINES LAYER ######################################################################################

    map_transit_lines_gdf = transit_lines_gdf[transit_lines_gdf['scenario'] == scenario]

    # Add the GeoDataFrame to the folium map with the symbology
    for _, row in map_transit_lines_gdf.iterrows():
        if pd.isna(row['aMxM']):
            continue  # Skip this row if 'aMxM' is NaN

        style = get_style_by_aMxM(row['aMxM'])  # Ensure you're using the correct column ('aMxM')

        # Create a GeoJson feature for each row
        folium.GeoJson(
            {
                'type': 'Feature',
                'geometry': row['geometry'].__geo_interface__,  # Convert geometry to geo_interface format
                'properties': {  # Add properties to the GeoJson
                    'SEGID': row['SEGID'],
                    'aMxM': row['aMxM']
                }
            },
            style_function=lambda x, style=style: {
                'color': style['color'],
                'weight': style['width'],
                'opacity': 0.95,
                'zIndex': 10000  # Ensure polylines appear above polygons
            },
            #tooltip=folium.GeoJsonTooltip(fields=['SEGID', 'aMxM'], aliases=['Segment', 'Max aMxM']),
        ).add_to(maps[map_scenario])

    # Render the map and save it with a unique div ID using the 'get_root().render()' method
    #tabs_html += maps[map_scenario].get_root().render()
    map_html = maps[map_scenario]._repr_html_()

    # add iframe id so can access through javascript:
    map_html = map_html.replace('<iframe ', f'<iframe id="iframe_{map_scenario}" ')

    # 2. Insert the Leaflet.Sync script before the closing </head> tag in the map HTML
    sync_script = '&lt;script src=&quot;https://cdn.jsdelivr.net/gh/jieter/Leaflet.Sync/L.Map.Sync.js&quot;&gt;&lt;/script&gt;'

    # 3. Add the sync script before the closing &lt;/head&gt; tag
    map_html = map_html.replace('&lt;/head&gt;', f'{sync_script}&lt;/head&gt;')

    # Extract the generated map id (something like 'map_ecc77ac144a2be94ec9eb5890258b06b')
    # Using regex to find the generated id
    import re
    map_id_match = re.search(r'map_\w+ \{', map_html)
    if map_id_match:
        trimmed_map_id_match = map_id_match.group()[:-2]
        print(trimmed_map_id_match)
        map_ids.append(trimmed_map_id_match)  # Store the id for later use

    # make variable globally accessible
    map_html = re.sub(r'var (map_\w+) =', r'window.\1 =', map_html)
    
    # get rid of warning: Make this Notebook Trusted to load map: File -> Trust Notebook
    map_html = map_html.replace('Make this Notebook Trusted to load map: File -> Trust Notebook','')


    # 1. Decode the map_html (if it's using HTML-encoded entities)
    #map_html = html.unescape(map_html)


    tabs_html += map_html

    # Close the tab content div
    tabs_html += '</div>\n'

# After rendering all maps, add the JavaScript code to synchronize them
if len(map_scenarios) > 1:
    # Add the leaflet.sync script
    sync_script = '''
    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css" />

    <script src="https://cdn.jsdelivr.net/gh/jieter/Leaflet.Sync/L.Map.Sync.js"></script>
    <script>
    document.addEventListener("DOMContentLoaded", function() {
        setTimeout(function() {
    '''

    if len(map_scenarios)==6:
        scenario0 = map_scenarios[0]
        scenario1 = map_scenarios[1]
        scenario2 = map_scenarios[2]
        scenario3 = map_scenarios[3]
        scenario4 = map_scenarios[4]
        scenario5 = map_scenarios[5]

        map_id0 = map_ids[0]
        map_id1 = map_ids[1]
        map_id2 = map_ids[2]
        map_id3 = map_ids[3]
        map_id4 = map_ids[4]
        map_id5 = map_ids[5]

        # Generate iframe IDs based on the scenario names
        sync_script += f'var iframe0 = document.getElementById("iframe_{scenario0}");\n'
        sync_script += f'var iframe1 = document.getElementById("iframe_{scenario1}");\n'
        sync_script += f'var iframe2 = document.getElementById("iframe_{scenario2}");\n'
        sync_script += f'var iframe3 = document.getElementById("iframe_{scenario3}");\n'
        sync_script += f'var iframe4 = document.getElementById("iframe_{scenario4}");\n'
        sync_script += f'var iframe5 = document.getElementById("iframe_{scenario5}");\n'

        # Access the map objects inside the iframes' contentWindow using map_ids
        sync_script += f'var map0 = iframe0.contentWindow.{map_id0};\n'
        sync_script += f'var map1 = iframe1.contentWindow.{map_id1};\n'
        sync_script += f'var map2 = iframe2.contentWindow.{map_id2};\n'
        sync_script += f'var map3 = iframe3.contentWindow.{map_id3};\n'
        sync_script += f'var map4 = iframe4.contentWindow.{map_id4};\n'
        sync_script += f'var map5 = iframe5.contentWindow.{map_id5};\n'

        # Synchronize the maps between the iframes
        sync_script += f'if (map0 && map1 && map2 && map3 && map4 && map5) {{\n'
        sync_script += f'    map0.sync(map0);\n'
        sync_script += f'    map1.sync(map0);\n'
        sync_script += f'    map2.sync(map0);\n'
        sync_script += f'    map3.sync(map0);\n'
        sync_script += f'    map4.sync(map0);\n'
        sync_script += f'    map5.sync(map0);\n'
        sync_script += f'    map0.sync(map1);\n'
        sync_script += f'    map1.sync(map1);\n'
        sync_script += f'    map2.sync(map1);\n'
        sync_script += f'    map3.sync(map1);\n'
        sync_script += f'    map4.sync(map1);\n'
        sync_script += f'    map5.sync(map1);\n'
        sync_script += f'    map0.sync(map2);\n'
        sync_script += f'    map1.sync(map2);\n'
        sync_script += f'    map2.sync(map2);\n'
        sync_script += f'    map3.sync(map2);\n'
        sync_script += f'    map4.sync(map2);\n'
        sync_script += f'    map5.sync(map2);\n'
        sync_script += f'    map0.sync(map3);\n'
        sync_script += f'    map1.sync(map3);\n'
        sync_script += f'    map2.sync(map3);\n'
        sync_script += f'    map3.sync(map3);\n'
        sync_script += f'    map4.sync(map3);\n'
        sync_script += f'    map5.sync(map3);\n'
        sync_script += f'    map0.sync(map4);\n'
        sync_script += f'    map1.sync(map4);\n'
        sync_script += f'    map2.sync(map4);\n'
        sync_script += f'    map3.sync(map4);\n'
        sync_script += f'    map4.sync(map4);\n'
        sync_script += f'    map5.sync(map4);\n'
        sync_script += f'    map0.sync(map5);\n'
        sync_script += f'    map1.sync(map5);\n'
        sync_script += f'    map2.sync(map5);\n'
        sync_script += f'    map3.sync(map5);\n'
        sync_script += f'    map4.sync(map5);\n'
        sync_script += f'    map5.sync(map5);\n'
        sync_script += f'    map0.sync(map6);\n'
        sync_script += f'    map1.sync(map6);\n'
        sync_script += f'    map2.sync(map6);\n'
        sync_script += f'    map3.sync(map6);\n'
        sync_script += f'    map4.sync(map6);\n'
        sync_script += f'    map5.sync(map6);\n'
        sync_script += f'}} else {{\n'
        sync_script += f'    console.error("Maps inside iframes for {scenario0} and {scenario1} and {scenario2}  and {scenario3}  and {scenario4}  and {scenario5}  are not properly initialized.");\n'
        sync_script += f'}}\n'

    sync_script += '''
        }, 6000);  // Delay by 6 seconds to ensure maps are loaded inside iframes
    });
    </script>
    '''

    tabs_html += sync_script
    print(sync_script)

# Add JavaScript to reload maps on tab click for proper rendering
tabs_html += '''
<script>

function openScenario(evt, scenarioName) {
    var i, tabcontent, tablinks;

    // Hide all tab contents
    tabcontent = document.getElementsByClassName("tabcontent");
    for (i = 0; i < tabcontent.length; i++) {
        tabcontent[i].classList.remove("active");
    }

    // Remove the active class from all tab links
    tablinks = document.getElementsByClassName("tablinks");
    for (i = 0; i < tablinks.length; i++) {
        tablinks[i].className = tablinks[i].className.replace(" active", "");
    }

    // Show the selected tab content and add the active class
    document.getElementById(scenarioName + "_container").classList.add("active");
    evt.currentTarget.className += " active";

}

function toggleSidebar() {
    var sidebar = document.getElementById("mySidebar");
    var mapContainer = document.getElementById("map-container");

    if (sidebar.style.width === "0px" || sidebar.style.width === "") {
        sidebar.style.width = "250px";  // Open the sidebar
        mapContainer.style.marginRight = "250px";  // Slide the map over
    } else {
        sidebar.style.width = "0";  // Close the sidebar
        mapContainer.style.marginRight = "0";  // Reset the map position
    }
}

// Open the sidebar by default when the page loads
window.onload = function() {
    var sidebar = document.getElementById("mySidebar");
    if (sidebar.style.width === "" || sidebar.style.width === "0px") {
        toggleSidebar();  // Open the sidebar if it's not open
    }
};

</script>

</body>
</html>
'''

# Save the entire HTML content to a file
with open('results/transit_maps_with_tabs.html', 'w') as file:
    file.write(tabs_html)


map_302d9b9ac3e141dd4e2299e9865587d0
map_4524b9b4afe66f6552c61c4eed36725f
map_4f0300cf082803bae740a7969fa0986e
map_061d55d394eb5ab92043448ad72972d1
map_0011f1e1e6999bec9edeed3ec5f7e51c
map_387e8859e4e01f0e58fd4cd0a0dd6092

    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css" />

    <script src="https://cdn.jsdelivr.net/gh/jieter/Leaflet.Sync/L.Map.Sync.js"></script>
    <script>
    document.addEventListener("DOMContentLoaded", function() {
        setTimeout(function() {
    var iframe0 = document.getElementById("iframe_oy-2023-p");
var iframe1 = document.getElementById("iframe_oy-2023-a");
var iframe2 = document.getElementById("iframe_oy-2023-od");
var iframe3 = document.getElementById("iframe_rtp-2050-p");
var iframe4 = document.getElementById("iframe_rtp-2050-a");
var iframe5 = document.getElementById("iframe_rtp-2050-od");
var map0 = iframe0.contentWindo